In [ ]:
import warnings
warnings.filterwarnings("ignore")

# pyAFQ: Automated Fiber Quantification ... in Python

## This notebook is available at: https://github.com/36000/babyAFQ_example


## Section 1: Four benefits of using pyAFQ compared to matlab AFQ:

### 1. Free and open source
    1. No need for a MATLAB license

### 2. Easier to run on cloud services
    1. For example, Amazon Web Services (AWS) or Microsoft Azure
    2. pyAFQ's only dependencies are other python packages which are installed automatically using "Package Installer for Python (pip)"
![](install_diagram.png)

### 3. Easier to run on BIDS datasets
1. The Brain Imaging Data Structure (BIDS) is a way of organizing neuroimaging and behavioral data
2. BIDS is a community effort
3. There are many [benefits to BIDS](https://bids.neuroimaging.io/benefits.html):
    1. It will be easy for another researcher to work on your data
        1. For example, by using BIDS you will save time trying to understand and reuse data acquired by a graduate student or postdoc that has already left the lab.
        
    2. There is a growing number of data analysis software that understand BIDS data, like pyAFQ
    
    3. Databases such as OpenNeuro.org, LORIS, COINS, XNAT, SciTran, and others will accept and export datasets organized according to BIDS.
        1. If you ever plan to share your data publicly (some journals require this) you can speed up the curation process by using BIDS.
        
    4. There are validation tools (also available online) that can check your dataset integrity and let you easily spot missing values.
    
![BIDS diagram](bids_diagram.png)

Image source: http://bids.neuroimaging.io/

### 4. Easier to customize methods and integrate with other pipelines
    1. pyAFQ provides tools to import tractography or mori group files from the original Matlab based AFQ to pyAFQ
    2. pyAFQ can use tractography from other pipelines like MRtrix3 or Qsiprep which implement other algorithms
        like SIFT and ACT

#### For babyAFQ, Use MRtrix3 for the tractography: https://www.mrtrix.org/

## Section 2: Baby pyAFQ code examples

### Step 1: Organizing the data
Put your dataset in BIDS. Here is an example dataset with one subject:
![Example BIDS](bids_example.png)

## This data is available at: https://tinyurl.com/35tptjz9

### Step 2: Initialize a GroupAFQ object
Creates a GroupAFQ object, that encapsulates tractometry. This object can be used to manage the entire AFQ pipeline, including:

    Tractography

    Registration

    Segmentation / Bundle Recognition

    Cleaning

    Profiling

    Visualization

In this case, MRtrix has already done the tractography (visualized using [MI-Brain](https://github.com/imeka/mi-brain)): ![mrtrix_track](3d_screenshot.png)

In [ ]:
from AFQ.api.group import GroupAFQ
import AFQ.api.bundle_dict as abd
import AFQ.data.fetch as afd
from AFQ.definitions.image import LabelledImageFile
import plotly

myafq = GroupAFQ(
    # ==== BIDS parameters ====
    bids_path="example_bids_subject",
    preproc_pipeline="vistasoft",
    # ===== Registration parameters ====
    reg_template_spec=afd.read_pediatric_templates()["UNCNeo-withCerebellum-for-babyAFQ"],
    reg_subject_spec="b0",
    # ==== Bundle parameters ====
    bundle_info=abd.PediatricBundleDict(),
    # ==== Tracking parameters ====
    import_tract={
        "suffix": "tractography", "scope": "mrtrix"},
    # ==== Segmentation parameters ====
    segmentation_params={
        "filter_by_endpoints": False},
    clean_params={
        'distance_threshold': 4},
)

### Step 3: Run pyAFQ
After defining your pyAFQ API object, you can ask for the output of any step of the pipeline. It is common for users to just call export_all (for example, myafq.export_all()).

In [ ]:
myafq.export("template_xform")

On the left, the babyAFQ template from MNI space has been transformed into the subject's native space. On the right, the subject's b0 is shown for reference.
![warped_template](warped_template.png)

In [ ]:
bundle_html = myafq.export("all_bundles_figure")
plotly.io.show(bundle_html["01"][0])

## Section 3: Two other features

### 1. Outputs can be used as inputs to AFQ-Insight
[AFQ-Insight](https://github.com/richford/AFQ-Insight) is an open source, python based statistical learning library for tractometry. Here are some features:
1. Format tract profiles for the popular [scikit-learn](https://scikit-learn.org/stable/) machine learning python library
2. Harmonize tract profiles across multiple sites using combat
3. Make regression and classification pipelines tailored to tract profiles
4. Use neural networks on tract profiles for prediction

### 2. Using the pyAFQ configuration file
pyAFQ provides a [command-line interface (CLI)](https://yeatmanlab.github.io/pyAFQ/usage/index.html). After installing the software, and organizing the data, run:

    pyAFQ /path/to/config.toml

At minimum, the file should contain the BIDS path:

    [BIDS_PARAMS]
    bids_path = "path/to/study"

But additional configuration options can be provided. Here is an example configuration file to run babyAFQ:

    [BIDS_PARAMS]
    bids_path = "example_bids_subject"
    preproc_pipeline = "vistasoft"

    [DATA]
    reg_template_spec = "pediatric"
    reg_subject_spec = "b0"
    bundle_info="PediatricBundleDict()"

    [TRACTOGRAPHY_PARAMS]
    import_tract = "{'suffix': 'tractography', 'scope': 'mrtrix'}"

    [SEGMENTATION_PARAMS]
    filter_by_endpoints = false
    
    [CLEANING_PARAMS]
    distance_threshold = 4

## Section 4: Summary
    1. pyAFQ is free and open source (easy to run on the cloud)

    2. pyAFQ is easy to install and does not require knowledge of python to use

    3. pyAFQ uses BIDS datasets

    4. pyAFQ is customizable and integrates with other pipelines like MRtrix
  
Documentation: https://yeatmanlab.github.io/pyAFQ/ <br />
Code: https://github.com/yeatmanlab/pyAFQ <br />

# Thank you for listening!